In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

This script serves to just showcase the implementation of 2RDMs corresponding to 1RDMFTs. The 3 basis sets, $\{\chi\}$,$\{\phi\}$ and $\{\eta\}$ atre all required to switch back and forth between known quantities to test and make sure everything is correct. This script is really just there to further my understanding of the relationship between 1RDMFT, 2RDMs and their implementations.



The following function caculates the electron electron interactio  energy from the Fock basis representation of the 2RDM $\gamma_2$ in the follwong way:
$$E_{ee} = \sum_p \sum_q \sum_r \sum_s \gamma_{2_{pqrs}} \langle p q | r s \rangle$$

In [2]:
def ee_from_gamma_2(eri,dm2):
    E = 0 
    dim = eri.shape[0]
    for p in range(0,dim):
        for q in range(0,dim):
            for r in range(0,dim):
                for s in range(0,dim):
                    E+=eri[p,q,r,s]*dm2[p,q,r,s]
    return E

In [3]:
def HF_2RDM(n, M):
    # this form can only make sense for spin restriced orbitals 
    TWORDM = np.zeros((M,M,M,M))
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    if i==j and k==l:
                        TWORDM[i,j,k,l] = 4
                        # print(TWORDM[i,j,k,l])
                    if i==l and k==j:
                        TWORDM[i,j,k,l] -= 2
                        # print(TWORDM[i,j,k,l])
                    # print(i,j,k,l,TWORDM[i,j,k,l], (i==j and k==l), (i==l and k==j), n[i]*n[k]/4) 
                    TWORDM[i,j,k,l] = n[i]/2*n[k]/2*TWORDM[i,j,k,l]
    return TWORDM
# this is only non 0 if aa bb or  ab ba. wisdom of old tells, only one orbital is occupied. 
# one of the occupation numbers for each of the brackets otherwise the resis ls n_a n_a instead of n_a n_b, comapre HF energy

Hartree Fock $E_{HF}$ energy written as function of occupation numbers $\{n\}$ and natural orbitals $\{\eta \}$:
$$
\begin{split}
E_{HF}[n,\eta] & = \sum_{a=1}^M  \sum_{b=1}^M n_a n_b (2 [aa |bb] - [ab |ba])\\
\end{split}
$$


In [4]:
# for spinorbitals
def ONERDMFT_HF_energy(Fouridx,  n):
    energy = 0
    M = Fouridx.shape[0]
    for a in  range(0,M):
        for b in range(0,M):
            energy += n[a]/2*n[b]/2*(2*Fouridx[a,a,b,b] - Fouridx[a,b,b,a])
    return energy

In [5]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
mol.basis = "cc-pvdz"
#mol.basis = "6-31g" 
#mol.basis = "sto3g" 

mol.build()

In [6]:
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')
N = mol.nelec[0]

In [7]:
# Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

converged SCF energy = -2.85516047724274


np.float64(-2.85516047724274)

In [8]:
C = mf.mo_coeff
P=np.matmul(C[:,0:N],C[:,0:N].T)

In [9]:
# get operators in AO basis 
J = mf.get_j()
K = mf.get_k()
h = mf.get_hcore()
hf_1RDM = mf.make_rdm1() # -> AO basis
hf_2RDM = mf.make_rdm2() # -> AO basis

In [10]:
# get operators in MO basis 
h1 = C.T@h@C
J_MO = C.T@J@C
K_MO = C.T@K@C
hf_1RDM_MO = C.T@S.T@hf_1RDM@S@C


# Find electron-repulsion integrals (eri).
eri_MO = ao2mo.kernel(mol, mf.mo_coeff)
eri_MO = np.asarray(ao2mo.restore(1, eri_MO, mol.nao))

# eri_MO = eri.copy()
# hf_2RDM_MO = hf_2RDM.copy()
# for i in range(4):
#   eri_MO = np.tensordot(eri_MO, C, axes=1).transpose(3, 0, 1, 2)
#   hf_2RDM_MO = np.tensordot(hf_2RDM_MO, C, axes=1).transpose(3, 0, 1, 2)

In [11]:
occ, nao =  np.linalg.eigh(hf_1RDM_MO)

In [12]:
hf_1RDM_NAO = nao.T@hf_1RDM_MO@nao
J_NAO = nao.T@J_MO@nao
K_NAO = nao.T@K_MO@nao
h1_NAO = nao.T@h1@nao
eri_NAO = eri_MO.copy()
#hf_2RDM_NAO = hf_2RDM_MO.copy()
for i in range(4):
  eri_NAO = np.tensordot(eri_NAO, nao, axes=1).transpose(3, 0, 1, 2)
  #hf_2RDM_NAO = np.tensordot(hf_2RDM_NAO, C, axes=1).transpose(3, 0, 1, 2)

In [13]:
#AOs
print(np.trace(np.matmul(hf_1RDM,h)),1/2.*np.trace(np.matmul(hf_1RDM, J))-1/4.*np.trace(np.matmul(hf_1RDM, K)), 1/2*ee_from_gamma_2(eri,hf_2RDM))
#MOs 
print(np.trace(np.matmul(hf_1RDM_MO,h1)),1/2.*np.trace(np.matmul(hf_1RDM_MO, J_MO))-1/4.*np.trace(np.matmul(hf_1RDM_MO, K_MO)))#, ee_from_gamma_2(eri_MO,hf_2RDM_MO))
#NAOs
print(np.trace(np.matmul(hf_1RDM_NAO,h1_NAO)),1/2.*np.trace(np.matmul(hf_1RDM_NAO, J_NAO))-1/4.*np.trace(np.matmul(hf_1RDM_NAO, K_NAO)))#, ee_from_gamma_2(eri_NAO,hf_2RDM_NAO))

-3.882025102606505 1.0268646253637652 1.0268646253637648
-3.882025102606506 1.0268646253637654
-3.882025102606506 1.0268646253637654


In [14]:
# First create FCI solver with function fci.FCI and solve the FCI problem
cisolver = fci.FCI(mol, mf.mo_coeff)
e, fcivec = cisolver.kernel()

In [15]:
dm1, dm2 = cisolver.make_rdm12(fcivec, mf.mo_coeff.shape[1], mol.nelec)
fci_occ, fci_nao =  np.linalg.eigh(dm1)
print(.5*ee_from_gamma_2(eri_MO,dm2), e-np.trace(np.matmul(dm1,h1)))

0.9612496317190583 0.961249631719058


In [16]:
print(np.trace(np.matmul(dm1,h1)), np.trace(np.matmul(hf_1RDM_MO,h1)))

-3.8488444628099954 -3.882025102606506


In [17]:
eri_fciNAO = eri_MO.copy()
eri_NAO = eri_MO.copy()
dm2_NAO = dm2.copy()
for i in range(4):
  eri_fciNAO = np.tensordot(eri_fciNAO, fci_nao, axes=1).transpose(3, 0, 1, 2)
  eri_NAO = np.tensordot(eri_NAO, nao, axes=1).transpose(3, 0, 1, 2)
  dm2_NAO = np.tensordot(dm2_NAO, fci_nao, axes=1).transpose(3, 0, 1, 2)

In [18]:
print(.5*ee_from_gamma_2(eri_MO,dm2), e-np.trace(np.matmul(dm1,h1)))
print(ONERDMFT_HF_energy(eri_NAO, occ))

0.9612496317190583 0.961249631719058
1.0268646253637654


In [19]:
hf_dm2_fcinao = HF_2RDM(fci_occ, eri_NAO.shape[0])

In [20]:
hf_dm2_nao = HF_2RDM(occ, eri_NAO.shape[0])

In [21]:
print(.5*ee_from_gamma_2(eri_NAO,hf_dm2_nao), e-np.trace(np.matmul(dm1,h1)))

1.0268646253637654 0.961249631719058


In [22]:
print(.5*ee_from_gamma_2(eri_fciNAO,hf_dm2_fcinao), e-np.trace(np.matmul(dm1,h1)))

1.0341444370318436 0.961249631719058


In [23]:
print(np.trace(np.matmul(hf_1RDM,h)))
print(np.trace(np.matmul(dm1,h1)))


-3.882025102606505
-3.8488444628099954
